In [207]:
# prep imports
from google.cloud import bigquery
import requests
import datetime
import warnings
import pandas as pd
warnings.filterwarnings('ignore')


In [208]:
# Set up variables
project_id    = 'atscale-sales-demo'
dataset_id    = 'mm_dataset'
table_queries = 'twinpines_queries'
table_queries_full = project_id + '.' + dataset_id + '.' + table_queries

hostname = "https://twinpines.gotdns.ch"
bearer_token = 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJlRm9weXBTSERqUWlRUnAtSHRFeG5jLXZTcS1FMkZnMEg2cmt1cDVzZ2trIn0.eyJleHAiOjE3MzMzODgyMzIsImlhdCI6MTcyNTYyMzYyOCwiYXV0aF90aW1lIjoxNzI1NjEyMjMyLCJqdGkiOiJhMWUyZmI2My0zMWI3LTQ5MTYtODBjNS1mMzkyYWMyZDMxZTUiLCJpc3MiOiJodHRwczovL3R3aW5waW5lcy5nb3RkbnMuY2gvYXV0aC9yZWFsbXMvYXRzY2FsZSIsImF1ZCI6WyJyZWFsbS1tYW5hZ2VtZW50IiwiYnJva2VyIiwiYWNjb3VudCIsImF0c2NhbGUtcHVibGljLWFwaSJdLCJzdWIiOiIyMGJhNDZlNy04N2YxLTQ1MzQtYjJjZC0zZjg3NjMzNDU3ZWUiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJhdHNjYWxlLWFwaSIsInNlc3Npb25fc3RhdGUiOiI2ZDhiNjVkNS1lZmRlLTQ4NGMtOWNjMC00Mzk1MDk0MzhlNTEiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbIi8qIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJyZXBvc2l0b3J5X3Byb2plY3RfcmVhZCIsImFnZ3JlZ2F0ZXNfdmlldyIsImRhdGF3YXJlaG91c2VzX2FkbWluIiwiYWdncmVnYXRlc19tYW5hZ2UiLCJxdWVyaWVzX21hbmFnZSIsImFkbWluIiwiZGVmYXVsdC1yb2xlcy1hdHNjYWxlIiwicmVwb3NpdG9yeV9wcm9qZWN0X3B1Ymxpc2giLCJpbXBlcnNvbmF0aW9uX3VzZXIiLCJzdXBlcnVzZXJfdXNlciIsInF1ZXJ5X2RhdGFzZXRfYXBpX3ZpZXciLCJkZXNpZ25jZW50ZXJfdXNlciIsIm9mZmxpbmVfYWNjZXNzIiwicXVlcnlfdXNlciIsInN1cHBvcnRfbG9nc192aWV3IiwicXVlcmllc192aWV3Il19LCJyZXNvdXJjZV9hY2Nlc3MiOnsicmVhbG0tbWFuYWdlbWVudCI6eyJyb2xlcyI6WyJ2aWV3LWlkZW50aXR5LXByb3ZpZGVycyIsInZpZXctcmVhbG0iLCJtYW5hZ2UtaWRlbnRpdHktcHJvdmlkZXJzIiwicmVhbG0tYWRtaW4iLCJjcmVhdGUtY2xpZW50IiwibWFuYWdlLXVzZXJzIiwicXVlcnktcmVhbG1zIiwidmlldy1hdXRob3JpemF0aW9uIiwicXVlcnktY2xpZW50cyIsInF1ZXJ5LXVzZXJzIiwibWFuYWdlLWV2ZW50cyIsIm1hbmFnZS1yZWFsbSIsInZpZXctZXZlbnRzIiwidmlldy11c2VycyIsInZpZXctY2xpZW50cyIsIm1hbmFnZS1hdXRob3JpemF0aW9uIiwibWFuYWdlLWNsaWVudHMiLCJxdWVyeS1ncm91cHMiXX0sImJyb2tlciI6eyJyb2xlcyI6WyJyZWFkLXRva2VuIl19LCJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJ2aWV3LXByb2ZpbGUiXX19LCJzY29wZSI6InByb2ZpbGUgeG1sYV90b2tlbiBlbWFpbCIsInNpZCI6IjZkOGI2NWQ1LWVmZGUtNDg0Yy05Y2MwLTQzOTUwOTQzOGU1MSIsImVtYWlsX3ZlcmlmaWVkIjpmYWxzZSwibmFtZSI6Ik1hcmNvIE3DvGxsZXIiLCJ4bWxhX3Rva2VuIjoiMjBiYTQ2ZTc4N2YxNDUzNGIyY2QzZjg3NjMzNDU3ZWUyMzUzYzE5YzM0YWVhNzY0ZGQ3ODc4MWQ2MTE4YzY1MiIsInByZWZlcnJlZF91c2VybmFtZSI6Im1hcmNvLm11bGxlckBhdHNjYWxlLXNlLWRlbW8uY29tIiwiZ2l2ZW5fbmFtZSI6Ik1hcmNvIiwiZmFtaWx5X25hbWUiOiJNw7xsbGVyIiwiZW1haWwiOiJtYXJjby5tdWxsZXJAYXRzY2FsZS1zZS1kZW1vLmNvbSJ9.WIAACSF_-Y2NNVED6iIDrF43KReccxfVSt-8dLa8-Rbyfcm8srXm72fwqi4aEFs5AClNDXefOdUAWuaks0JFhjS1ypVD5crfjB04c0MTDnXeSR_u27coRHFCmpbz7QknWm87vahuI-akhgQHRfu--PZoqv4PQlr4bA_uvf-QAhUglHkiRPEAF8krzzoJFsnragCQ8pAAFugTC_Nv9d7wKiI1pTkc-85V2oqc9ejfUTW95QOcqWwnaqXVSLc3T09MVOCwuRgokUKm2DcYFj6CGeheSH6B6ZZ5AvNXxDlksqcoqWvuBg9fA24J791CDc7UJUcsz12ugXJ4FWcX_yNbEg'
full_load_hours = 167 #max limit from atscale <168

In [209]:
#Create GBQ Client
client = bigquery.Client()

In [210]:
#get last load max datetime

sql_query = f'SELECT main_startTime FROM `{table_queries_full}` order by main_startTime DESC limit 1;'

query_job = client.query(sql_query)
try:
  results = query_job.result()
  for row in results:
    lastload = row

except:
  lastload = None
lastload

Row((datetime.datetime(2024, 9, 6, 12, 15, 38, 237000, tzinfo=datetime.timezone.utc),), {'main_startTime': 0})

In [211]:
if not(lastload):
  lastload = str(datetime.datetime.now() - datetime.timedelta(hours=full_load_hours))
else:
  lastload = str(lastload['main_startTime'].strftime("%Y-%m-%d, %H:%M:%S.%f"))


In [212]:
#load log from atscale
url = hostname + "/api/queries?showCanaries=false&startDate="+ lastload
print(url)
headers = {
  'Accept': 'application/json',
  'Authorization': 'Bearer ' + bearer_token
}

records = []
# Start at Page 1
current_page = 1

# Loop through all the pages
while True:
    response = requests.request("GET", url, headers=headers, verify=False, params={'page': current_page })
    if response.status_code <= 299:
      records += response.json()['results']
    else:
      print(response.text)
      raise SystemExit("Stop notebook due to API error")

    # End loop if the current page we requested is the same as the last page the API returns
    if current_page == response.json()['totalPages']:
        break

    # Increment the page to request
    #current_page =response.json()['totalPages']
    current_page += 1

records


https://twinpines.gotdns.ch/api/queries?showCanaries=false&startDate=2024-09-06, 12:15:38.237000


[{'optimization': ['AGGS'],
  'attributes': ['CountryCity'],
  'measures': ['orderquantity1'],
  'traceId': 'a4d05bc061afbd98d20c5524322eac34',
  'queryId': '0e9846f2-4d68-4e85-b4a6-77cad990680b',
  'status': 'successful',
  'startTime': 1725624938237,
  'duration': 1321,
  'modelId': 'f76fbada-a7af-5a2e-adea-529236c6323d',
  'catalogId': 'dd2ac454-5a72-50f7-8f73-11bec7a0f447',
  'userId': 'marco.muller@atscale-se-demo.com',
  'queryType': 'User',
  'events': [{'name': 'Inbound Query',
    'startTime': 1725624938237,
    'duration': 43,
    'queryId': '0e9846f2-4d68-4e85-b4a6-77cad990680b'},
   {'name': 'Planning', 'startTime': 1725624938280, 'duration': 246},
   {'name': 'Outbound',
    'startTime': 1725624939081,
    'duration': 475,
    'subqueries': [{'name': 'Query 1',
      'startTime': 1725624939081,
      'duration': 475,
      'subqueryId': '0e9846f2-4d68-4e85-b4a6-77cad990680b'}]},
   {'name': 'Result Processing', 'startTime': 1725624939556, 'duration': 1}],
  'catalogName': 

In [213]:
%tb

SystemExit: Stop notebook due to API error

In [214]:
# create flat structure from recods -> events -> subqueries

rows = []
main_fields = ['traceId','queryId','startTime','duration','modelId','catalogId','userId','queryType','catalogName','modelName','user']
subquery_fields = ['name','duration','subqueryId']
# appending rows
for data in records:
  events = data['events']
  print(events)
  for event in events:
    for main_field in main_fields:
      try:
        event['main_'+main_field] = data[main_field]
      except:
        break

    if'subqueries' in event.keys():
      subqueries = event['subqueries']
      for subquery in subqueries:
        for subquery_field in subquery_fields:
          try:
            event['subquery_'+subquery_field] = subquery[subquery_field]
          except:
            break

    else:
      event['subquery_subqueryId'] = None

    rows.append(event)


df = pd.DataFrame(rows)

#define the keys to remove
df.drop(columns=['subqueries'], inplace=True)
df.rename(columns={'name': 'event_name', 'startTime': 'event_startTime', 'duration': 'event_duration', 'queryId': 'event_queryId'}, inplace=True)
df['main_startTime'] = df['main_startTime'].apply(lambda x:datetime.datetime.fromtimestamp(x/1000.0))
df['event_startTime'] = df['event_startTime'].apply(lambda x:datetime.datetime.fromtimestamp(x/1000.0))


[{'name': 'Inbound Query', 'startTime': 1725624938237, 'duration': 43, 'queryId': '0e9846f2-4d68-4e85-b4a6-77cad990680b'}, {'name': 'Planning', 'startTime': 1725624938280, 'duration': 246}, {'name': 'Outbound', 'startTime': 1725624939081, 'duration': 475, 'subqueries': [{'name': 'Query 1', 'startTime': 1725624939081, 'duration': 475, 'subqueryId': '0e9846f2-4d68-4e85-b4a6-77cad990680b'}]}, {'name': 'Result Processing', 'startTime': 1725624939556, 'duration': 1}]
[{'name': 'Inbound Query', 'startTime': 1725624937765, 'duration': 167, 'queryId': 'dafb3ffc-69e9-4156-9335-2dc89889e729'}, {'name': 'Planning', 'startTime': 1725624937932, 'duration': 257}, {'name': 'Outbound', 'startTime': 1725624938469, 'duration': 399, 'subqueries': [{'name': 'Query 1', 'startTime': 1725624938469, 'duration': 399, 'subqueryId': 'dafb3ffc-69e9-4156-9335-2dc89889e729'}]}, {'name': 'Result Processing', 'startTime': 1725624938868, 'duration': 3}]


In [215]:
df

,event_name,event_startTime,event_duration,event_queryId,main_traceId,main_queryId,main_startTime,main_duration,main_modelId,main_catalogId,main_userId,main_queryType,main_catalogName,main_modelName,main_user,subquery_subqueryId,subquery_name,subquery_duration
0,Inbound Query,2024-09-06 12:15:38.237,43,0e9846f2-4d68-4e85-b4a6-77cad990680b,a4d05bc061afbd98d20c5524322eac34,0e9846f2-4d68-4e85-b4a6-77cad990680b,2024-09-06 12:15:38.237,1321,f76fbada-a7af-5a2e-adea-529236c6323d,dd2ac454-5a72-50f7-8f73-11bec7a0f447,marco.muller@atscale-se-demo.com,User,Sales Insights - Snowflake_main,Internet Sales,Marco Müller,None,NaN,NaN
1,Planning,2024-09-06 12:15:38.280,246,NaN,a4d05bc061afbd98d20c5524322eac34,0e9846f2-4d68-4e85-b4a6-77cad990680b,2024-09-06 12:15:38.237,1321,f76fbada-a7af-5a2e-adea-529236c6323d,dd2ac454-5a72-50f7-8f73-11bec7a0f447,marco.muller@atscale-se-demo.com,User,Sales Insights - Snowflake_main,Internet Sales,Marco Müller,None,NaN,NaN
2,Outbound,2024-09-06 12:15:39.081,475,NaN,a4d05bc061afbd98d20c5524322eac34,0e9846f2-4d68-4e85-b4a6-77cad990680b,2024-09-06 12:15:38.237,1321,f76fbada-a7af-5a2e-adea-529236c6323d,dd2ac454-5a72-50f7-8f73-11bec7a0f447,marco.muller@atscale-se-demo.com,User,Sales Insights - Snowflake_main,Internet Sales,Marco Müller,0e9846f2-4d68-4e85-b4a6-77cad990680b,Query 1,475.0
3,Result Processing,2024-09-06 12:15:39.556,1,NaN,a4d05bc061afbd98d20c5524322eac34,0e9846f2-4d68-4e85-b4a6-77cad990680b,2024-09-06 12:15:38.237,1321,f76fbada-a7af-5a2e-adea-529236c6323d,dd2ac454-5a72-50f7-8f73-11bec7a0f447,marco.muller@atscale-se-demo.com,User,Sales Insights - Snowflake_main,Internet Sales,Marco Müller,None,NaN,NaN
4,Inbound Query,2024-09-06 12:15:37.765,167,dafb3ffc-69e9-4156-9335-2dc89889e729,94afa684ef505f5dd1d466c6137fbb60,dafb3ffc-69e9-4156-9335-2dc89889e729,2024-09-06 12:15:37.765,1106,f76fbada-a7af-5a2e-adea-529236c6323d,dd2ac454-5a72-50f7-8f73-11bec7a0f447,marco.muller@atscale-se-demo.com,User,Sales Insights - Snowflake_main,Internet Sales,Marco Müller,None,NaN,NaN
5,Planning,2024-09-06 12:15:37.932,257,NaN,94afa684ef505f5dd1d466c6137fbb60,dafb3ffc-69e9-4156-9335-2dc89889e729,2024-09-06 12:15:37.765,1106,f76fbada-a7af-5a2e-adea-529236c6323d,dd2ac454-5a72-50f7-8f73-11bec7a0f447,marco.muller@atscale-se-demo.com,User,Sales Insights - Snowflake_main,Internet Sales,Marco Müller,None,NaN,NaN
6,Outbound,2024-09-06 12:15:38.469,399,NaN,94afa684ef505f5dd1d466c6137fbb60,dafb3ffc-69e9-4156-9335-2dc89889e729,2024-09-06 12:15:37.765,1106,f76fbada-a7af-5a2e-adea-529236c6323d,dd2ac454-5a72-50f7-8f73-11bec7a0f447,marco.muller@atscale-se-demo.com,User,Sales Insights - Snowflake_main,Internet Sales,Marco Müller,dafb3ffc-69e9-4156-9335-2dc89889e729,Query 1,399.0
7,Result Processing,2024-09-06 12:15:38.868,3,NaN,94afa684ef505f5dd1d466c6137fbb60,dafb3ffc-69e9-4156-9335-2dc89889e729,2024-09-06 12:15:37.765,1106,f76fbada-a7af-5a2e-adea-529236c6323d,dd2ac454-5a72-50f7-8f73-11bec7a0f447,marco.muller@atscale-se-demo.com,User,Sales Insights - Snowflake_main,Internet Sales,Marco Müller,None,NaN,NaN


In [216]:
#Create Big Query Schema
from google.cloud import bigquery
from google.cloud.bigquery.schema import SchemaField
from typing import List

def generate_bigquery_schema(df: pd.DataFrame) -> List[SchemaField]:
    TYPE_MAPPING = {
        "i": "INTEGER",
        "u": "NUMERIC",
        "b": "BOOLEAN",
        "f": "FLOAT",
        "O": "STRING",
        "S": "STRING",
        "U": "STRING",
        "M": "TIMESTAMP",
    }
    schema = []
    for column, dtype in df.dtypes.items():
        val = df[column].iloc[0]
        mode = "REPEATED" if isinstance(val, list) else "NULLABLE"

        if isinstance(val, dict) or (mode == "REPEATED" and isinstance(val[0], dict)):
            fields = generate_bigquery_schema(pd.json_normalize(val))
        else:
            fields = ()

        type = "RECORD" if fields else TYPE_MAPPING.get(dtype.kind)
        schema.append(
            SchemaField(
                name=column,
                field_type=type,
                mode=mode,
                fields=fields,
            )
        )
    return schema
bigquery_schema = generate_bigquery_schema(df)


In [217]:
#Write query log records to gbq

job_config = bigquery.LoadJobConfig(
    # Specify a (partial) schema. All columns are always written to the
    # table. The schema is used to assist in data type definitions.
    schema=bigquery_schema,
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    write_disposition="WRITE_APPEND",
)

job = client.load_table_from_dataframe(

    df, table_queries_full, job_config=job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.

table = client.get_table(table_queries_full)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)


Loaded 2864 rows and 18 columns to atscale-sales-demo.mm_dataset.twinpines_queries


In [218]:
del df